In [79]:
from neo4j import GraphDatabase
from random import randint,choice
import datetime

In [2]:
conn = GraphDatabase.driver(uri = "bolt://localhost:7687",auth=("vishal",'123456'))

In [3]:
print(conn)

In [4]:
conn.address

IPv4Address(('localhost', 7687))

In [5]:
conn.session()

In [6]:
conn.verify_connectivity

<bound method experimental.<locals>.f__.<locals>.f_ of <neo4j.BoltDriver object at 0x000001DB68924CC8>>

In [7]:
conn.verify_connectivity()

G:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: ExperimentalWarning: The configuration may change in the future.
  """Entry point for launching an IPython kernel.


'Neo4j/4.2.1'

In [8]:
conn.session

<bound method BoltDriver.session of <neo4j.BoltDriver object at 0x000001DB68924CC8>>

In [13]:
sess = conn.session(database="python3")

In [14]:
sess.run("CREATE(a:Product{ProdName:'Pen'})")

In [15]:
sess.run("CREATE(s:Supplier{SuppName :'Reynolds'})")

In [16]:
sess.run("MATCH(a:Product{ProdName:'Pen'}) \
         MATCH(s:Supplier{SuppName :'Reynolds'}) \
         CREATE (s)-[:SUPPLY]->(a)\
         ")

In [22]:
sess.run("CREATE(a:Product{ProdName:'Printer'})")

In [23]:
sess.run("CREATE(s:Supplier{SuppName :'Brother'})")

In [24]:
sess.run("MATCH(a:Product{ProdName:'Printer'}) \
         MATCH(s:Supplier{SuppName :'Brother'}) \
         CREATE (s)-[:SUPPLY]->(a)\
         ")

In [25]:
sess.run("MATCH(a:Product{ProdName:'Printer'}) \
         MERGE(s:Supplier{SuppName :'Cannon'}) \
         CREATE (s)-[:SUPPLY]->(a)\
         ")

In [26]:
sess.run("MATCH(a:Product{ProdName:'Printer'}) \
         MERGE(s:Supplier{SuppName :'HP'}) \
         CREATE (s)-[:SUPPLY]->(a)\
         ")

In [27]:
sess.run("MERGE(a:Product{ProdName:'File Cover'}) \
         MERGE(s:Supplier{SuppName :'up&up'}) \
         CREATE (s)-[:SUPPLY]->(a)\
         ")

In [28]:
sess.run("MERGE(a:Product{ProdName:'Envelopes'}) \
         MERGE(s:Supplier{SuppName :'Prince'}) \
         CREATE (s)-[:SUPPLY]->(a)\
         ")

In [29]:
sess.run("MERGE(a:Product{ProdName:'Envelopes'}) \
         MERGE(s:Supplier{SuppName :'Prince'}) \
         CREATE (s)-[:SUPPLY]->(a)\
         ")

In [30]:
sess.run("MERGE(a:Product{ProdName:'Markers'}) \
         MERGE(s:Supplier{SuppName :'Reynolds'}) \
         CREATE (s)-[:SUPPLY]->(a)\
         ")

In [33]:
sess.run("Create(c:Category{CategoryNm : 'Stationary'})")

In [34]:
sess.run("MATCH(c:Category) \
          MATCH (p:Product)\
          Create (p)-[:HAS_CATEGORY]->(c)\
          "
        )

In [35]:
prod_list = ['Printer','Pen','Markers','File Cover','Notebooks','Envelopes']

In [124]:
cust_list =['WF','BoA','JPM','Citi','USB','5/3rd','FirstCitizen','Walmart','Kroger','HT','CTS','TIAA']

In [81]:
sess.run("CREATE(a:Customer{CompanyName:'WF'})")

In [82]:
sess.run("CREATE(a:Customer{CompanyName:'BoA'})")

In [83]:
sess.run("CREATE(a:Customer{CompanyName:'JPM'})")
sess.run("CREATE(a:Customer{CompanyName:'Citi'})")
sess.run("CREATE(a:Customer{CompanyName:'USB'})")

In [84]:
sess.run("CREATE(a:Customer{CompanyName:'5/3rd'})")
sess.run("CREATE(a:Customer{CompanyName:'FirstCitizen'})")
sess.run("CREATE(a:Customer{CompanyName:'Walmart'})")
sess.run("CREATE(a:Customer{CompanyName:'Kroger'})")
sess.run("CREATE(a:Customer{CompanyName:'HT'})")
sess.run("CREATE(a:Customer{CompanyName:'CTS'})")
sess.run("CREATE(a:Customer{CompanyName:'TIAA'})")

In [59]:
%%time
prod=choice(prod_list)
print(prod)
res = sess.run("MATCH (p:Product) RETURN p.ProdName as ProdName")
for record in res:
    print(record["ProdName"])

Notebooks
Pen
Printer
File Cover
Envelopes
Notebooks
Markers
Wall time: 2.98 ms


In [87]:
prod=choice(prod_list)
print(prod)
res = sess.run("MATCH (p:Product{ProdName : $prod}) RETURN p.ProdName as ProdName",prod=prod) # Need better understanding of the query
for record in res:
    print(record["ProdName"])

Pen
Pen


In [86]:
prod=choice(prod_list)
print(prod)

Envelopes


In [134]:
for order in range(100050,100052):
    #print('Order_id : ' + str(order))
    prod=choice(prod_list)
    #print(prod)
    
    start_date = datetime.date(2020, 1, 1)
    random_days = randint(1,364)
    Order_date = start_date + datetime.timedelta(days=random_days)
    #print(Order_date)
    
    qty=randint(1,100)
    #print(qty)
    
    CompanyName=choice(cust_list)
    
    
    line_item = randint(1,6)
    if line_item==1:
        #Insert Data into Graph
        sess.run("MATCH (p:Product{ProdName : $prod})\
                  MATCH (c:Customer{CompanyName : $CompanyName})\
                  MERGE(o:Order{OrderId:$order,OrderDt:$Order_date }) \
                  MERGE (c)-[:PLACED_ORDER]->(o)-[:HAS_PROD{Qty:$qty}]->(p)\
                 ",
                 prod=prod,
                 order=order,
                 Order_date=Order_date,
                 CompanyName=CompanyName,
                 qty=qty
            )
        print(str(order) +' '+CompanyName +' '+prod+' '+str(Order_date)+' '+str(qty))
    else:
        #Insert Data into Graph
        sess.run("MATCH (c:Customer{CompanyName : $CompanyName})\
                  MERGE(o:Order{OrderId:$order,OrderDt:$Order_date }) \
                  MERGE (c)-[:PLACED_ORDER]->(o)\
                 ",                 
                 order=order,
                 Order_date=Order_date,
                 CompanyName=CompanyName
                )
        prod_tmp_lst =[]
        for i in range(1,line_item+1):
            prod=choice(prod_list)
            if prod in prod_tmp_lst:
                continue
            prod_tmp_lst.append(prod)
            qty=randint(1,100)
            #Insert Data into Graph
            sess.run("MATCH (p:Product{ProdName : $prod})\
                      MATCH (o:Order{OrderId:$order }) \
                      MERGE (o)-[:HAS_PROD{Qty:$qty}]->(p)\
                     ",
                     prod=prod,
                     order=order,
                     qty=qty
                )
            print('ELSE : Line_Item : '+str(i)+' '+str(order) +' '+CompanyName+' '+str(Order_date) +' '+prod+' '+str(qty))
    print('\n')
    

ELSE : Line_Item : 1 100050 FirstCitizen 2020-06-09 Notebooks 34
ELSE : Line_Item : 2 100050 FirstCitizen 2020-06-09 Pen 51
ELSE : Line_Item : 5 100050 FirstCitizen 2020-06-09 Envelopes 33


ELSE : Line_Item : 1 100051 Walmart 2020-10-19 Envelopes 60
ELSE : Line_Item : 2 100051 Walmart 2020-10-19 Printer 50
ELSE : Line_Item : 3 100051 Walmart 2020-10-19 Notebooks 46
ELSE : Line_Item : 4 100051 Walmart 2020-10-19 Markers 39




In [122]:
line_item = randint(1,6)
print(line_item)
for i in range(1,line_item):
    print('i = '+ str(i))

2
i = 1


In [128]:
lst=['a','b']

In [129]:
lst.append('c')

In [130]:
lst

['a', 'b', 'c']

In [131]:
'a' in lst

True

In [132]:
lst=[]

In [133]:
lst

[]